In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report
import torch

In [2]:
xlmr = AutoModelForSequenceClassification.from_pretrained('joeddav/xlm-roberta-large-xnli')

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
tokenizer = AutoTokenizer.from_pretrained('joeddav/xlm-roberta-large-xnli')

In [4]:
train_data = pd.read_csv("../data/train.csv")

In [5]:
test_data = pd.read_csv("../data/test.csv")

In [7]:
# match model labels and dataset labels
print(xlmr.config.label2id)
train_data['label'] = train_data['label'].replace([0, 2], [2, 0])
train_data.head(3)

{'contradiction': 0, 'entailment': 2, 'neutral': 1}


,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,2
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,0
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,2


In [8]:
train_data["both"] = "<s> "+train_data["premise"]+" </s> "+train_data["hypothesis"]
test_data["both"] = "<s> "+test_data["premise"]+" </s> "+test_data["hypothesis"]

In [9]:
train_data.head()

,id,premise,hypothesis,lang_abv,language,label,both
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,2,<s> and these comments were considered in form...
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,0,<s> These are issues that we wrestle with in p...
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,2,<s> Des petites choses comme celles-là font un...
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,2,<s> you know they can't really defend themselv...
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1,<s> ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได...


In [17]:
for i in range(10):
    print(len(tokenizer.encode(train_data["both"].values[i])))

34
38
39
36
55
53
73
30
44
42
